In [1]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imutils

In [5]:
def crop_brain_contour(image, plot=False):
    
    #import imutils
    #import cv2
    #from matplotlib import pyplot as plt
    
    # Convert the image to grayscale, and blur it slightly
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Threshold the image, then perform a series of erosions +
    # dilations to remove any small regions of noise
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    

    # Find the extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    
    # crop new image out of the original image using the four extreme points (left, right, top, bottom)
    new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]            

    if plot:
        plt.figure()

        plt.subplot(1, 2, 1)
        plt.imshow(image)
        
        plt.tick_params(axis='both', which='both', 
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)
        
        plt.title('Original Image')
            
        plt.subplot(1, 2, 2)
        plt.imshow(new_image)

        plt.tick_params(axis='both', which='both', 
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)

        plt.title('Cropped Image')
        
        plt.show()
    
    return new_image

In [8]:
def preprocess_user_input(image_path, image_size):
    """
    Preprocess the user input image. 
    Arguments:
        image_path: string representing the file path of the image.
        image_size: tuple representing the desired image size.
    Returns:
        img: A numpy array with shape = (image_width, image_height, #_channels)
    """

    # load the image
    image = cv2.imread("C:\\brain\\no\\14 no.jpg")
    
    # crop the brain and ignore the unnecessary rest part of the image
    image = crop_brain_contour(image, plot=False)
    
    # resize image
    image_width, image_height = image_size
    image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
    
    # normalize values
    image = image / 255.
    
    # convert image to numpy array and reshape it to be ready to use with the model
    img = np.array(image)
    img = img.reshape(1, image_width, image_height, 3)
    
    return img



In [9]:
# Load the saved model

model = load_model('c:\\brain\\models\\cnn-parameters-improvement-01-0.89.keras')
user_img = preprocess_user_input(img, (240, 240))
prediction = model.predict(user_img)

# Interpret the prediction
predict = (prediction[0][0] * 10)
print(predict)
result = ""
if predict >= 1:
    result = 'Yes, the model predicts that this image is of a brain tumor.'
else:
    result = 'No, the model predicts that this image is not of a brain tumor.'

print(result)

NameError: name 'img' is not defined

In [4]:
import os
from keras.models import load_model
import numpy as np

# Load the saved model
model = load_model('c:\\brain\\models\\cnn-parameters-improvement-04-0.90.keras')

def preprocess_user_input(image_path, image_size):
    """
    Preprocess the user input image. 
    Arguments:
        image_path: string representing the file path of the image.
        image_size: tuple representing the desired image size.
    Returns:
        img: A numpy array with shape = (image_width, image_height, #_channels)
    """

    # load the image
    image = cv2.imread(image_path)
    
    # crop the brain and ignore the unnecessary rest part of the image
    image = crop_brain_contour(image, plot=False)
    
    # resize image
    image_width, image_height = image_size
    image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
    
    # normalize values
    image = image / 255.
    
    # convert image to numpy array and reshape it to be ready to use with the model
    img = np.array(image)
    img = img.reshape(1, image_width, image_height, 3)
    
    return img

# Directory containing images
directory = 'C:\\brain\\Negative'

# Initialize counters
count_Y = 0
count_N = 0

# Loop through all images in the directory
for filename in os.listdir(directory):
    if filename.endswith(".jpg") :
        # Preprocess the image
        img_path = os.path.join(directory, filename)
        user_img = preprocess_user_input(img_path, (240, 240))

        # Make a prediction
        prediction = model.predict(user_img)

        # Interpret the prediction
        predict = (prediction[0][0] * 10)
        if predict >= 1:
            count_Y += 1
        else:
            count_N += 1

# Print the results
print("Number of tumorous images (Y):", count_Y)
print("Number of non-tumorous images (N):", count_N)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 584ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━